In [4]:
import os
from uuid import uuid1
from dotenv import load_dotenv, find_dotenv

unique_id = uuid1().hex[0:8]
_ = load_dotenv(find_dotenv())

In [5]:
from langsmith import Client

client = Client()

In [9]:
from operator import itemgetter

from langchain import hub
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
llm_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])

prompt = hub.pull("wfh/agent-lcel-prompt")

def format_scratchpad(x: dict) -> str:
    return format_to_openai_functions(x["intermediate_steps"])

def get_chat_history(x: dict) -> list:
    return x.get("chat_history", [])

agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": format_scratchpad,
        "chat_history": get_chat_history,
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
).with_config(run_name="Agent")

agent_executor = AgentExecutor(
    agent=agent, tools=tools, handle_parsing_errors=True
)

In [10]:
# result = agent_executor.invoke({"input": "how much larger is the biggest star compared to the sun?"})

In [11]:
# result

{'input': 'how much larger is the biggest star compared to the sun?',
 'output': 'The largest known star, UY Scuti, is estimated to be about 1,700 times larger in radius than the Sun. This makes it one of the largest stars known to us.'}